<a href="https://colab.research.google.com/github/aSafarpoor/Recommender_Systems/blob/main/NCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A pytorch GPU implementation of He et al. "Neural Collaborative Filtering**

based on https://github.com/hexiangnan/neural_collaborative_filtering

In [1]:
batch_size=256 
lr=0.001 
factor_num=16

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/MSc/codes

/content/drive/MyDrive/MSc/codes


# Config

In [4]:
%mkdir recommendation
%cd recommendation
%mkdir NCF-Data
%cd NCF-Data
!pwd

mkdir: cannot create directory ‘recommendation’: File exists
/content/drive/MyDrive/MSc/codes/recommendation
mkdir: cannot create directory ‘NCF-Data’: File exists
/content/drive/MyDrive/MSc/codes/recommendation/NCF-Data
/content/drive/MyDrive/MSc/codes/recommendation/NCF-Data


In [5]:
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.train.rating
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.rating
# !wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.negative

In [6]:
class Config:
    def __init__(self):
        # dataset name 
        self.dataset = 'ml-1m'

        # model name 
        self.model = 'NeuMF-end'

        # paths
        self.main_path = '/recommendation/NCF-Data/'

        self.train_rating = self.main_path + '{}.train.rating'.format(self.dataset)
        self.test_rating = self.main_path + '{}.test.rating'.format(self.dataset)
        self.test_negative = self.main_path + '{}.test.negative'.format(self.dataset)

        self.model_path = './models/'
        self.GMF_model_path = self.model_path + 'GMF.pth'
        self.MLP_model_path = self.model_path + 'MLP.pth'
        self.NeuMF_model_path = self.model_path + 'NeuMF.pth'

# Data utils:

In [7]:
import numpy as np 
import pandas as pd 
import scipy.sparse as sp
import torch.utils.data as data

config = Config()

In [8]:
!pwd
config.train_rating 

/content/drive/MyDrive/MSc/codes/recommendation/NCF-Data


'/recommendation/NCF-Data/ml-1m.train.rating'

In [9]:

def load_all(test_num=100):
    ''' We load all the three file here to save time in each epoch. '''
    train_data = pd.read_csv(
		'/content/drive/MyDrive/MSc/codes'+config.train_rating, 
		sep='\t', header=None, names=['user', 'item'], 
		usecols=[0, 1], dtype={0: np.int32, 1: np.int32})
    
    user_num = train_data['user'].max() + 1
    item_num = train_data['item'].max() + 1

    train_data = train_data.values.tolist()

    # load ratings as a dok matrix
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in train_data:
        train_mat[x[0], x[1]] = 1.0

    test_data = []
    with open('/content/drive/MyDrive/MSc/codes' + config.test_negative, 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t')
            u = eval(arr[0])[0]
            test_data.append([u, eval(arr[0])[1]])
            for i in arr[1:]:
                test_data.append([u, int(i)])
            line = fd.readline()
    return train_data, test_data, user_num, item_num, train_mat		

In [10]:
class NCFData(data.Dataset):
	def __init__(self, features, 
				num_item, train_mat=None, num_ng=0, is_training=None):
		super(NCFData, self).__init__()
		""" Note that the labels are only useful when training, we thus 
			add them in the ng_sample() function.
		"""
		self.features_ps = features
		self.num_item = num_item
		self.train_mat = train_mat
		self.num_ng = num_ng
		self.is_training = is_training
		self.labels = [0 for _ in range(len(features))]

	def ng_sample(self):
		assert self.is_training, 'no need to sampling when testing'

		self.features_ng = []
		for x in self.features_ps:
			u = x[0]
			for t in range(self.num_ng):
				j = np.random.randint(self.num_item)
				while (u, j) in self.train_mat:
					j = np.random.randint(self.num_item)
				self.features_ng.append([u, j])

		labels_ps = [1 for _ in range(len(self.features_ps))]
		labels_ng = [0 for _ in range(len(self.features_ng))]

		self.features_fill = self.features_ps + self.features_ng
		self.labels_fill = labels_ps + labels_ng

	def __len__(self):
		return (self.num_ng + 1) * len(self.labels)

	def __getitem__(self, idx):
		features = self.features_fill if self.is_training \
					else self.features_ps
		labels = self.labels_fill if self.is_training \
					else self.labels

		user = features[idx][0]
		item = features[idx][1]
		label = labels[idx]
		return user, item ,label

# evaluate

In [11]:
import numpy as np
import torch

In [12]:
def hit(gt_item, pred_items):
	if gt_item in pred_items:
		return 1
	return 0

In [13]:
def ndcg(gt_item, pred_items):
	if gt_item in pred_items:
		index = pred_items.index(gt_item)
		return np.reciprocal(np.log2(index+2))
	return 0

In [14]:
def metrics(model, test_loader, top_k):
	HR, NDCG = [], []

	for user, item, label in test_loader:
		user = user.cuda()
		item = item.cuda()

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		gt_item = item[0].item()
		HR.append(hit(gt_item, recommends))
		NDCG.append(ndcg(gt_item, recommends))

	return np.mean(HR), np.mean(NDCG)

# model:

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [16]:
class NCF(nn.Module):
	def __init__(self, user_num, item_num, factor_num, num_layers,
					dropout, model, GMF_model=None, MLP_model=None):
		super(NCF, self).__init__()
		"""
		user_num: number of users;
		item_num: number of items;
		factor_num: number of predictive factors;
		num_layers: the number of layers in MLP model;
		dropout: dropout rate between fully connected layers;
		model: 'MLP', 'GMF', 'NeuMF-end', and 'NeuMF-pre';
		GMF_model: pre-trained GMF weights;
		MLP_model: pre-trained MLP weights.
		"""		
		self.dropout = dropout
		self.model = model
		self.GMF_model = GMF_model
		self.MLP_model = MLP_model

		self.embed_user_GMF = nn.Embedding(user_num, factor_num)
		self.embed_item_GMF = nn.Embedding(item_num, factor_num)
		self.embed_user_MLP = nn.Embedding(
				user_num, factor_num * (2 ** (num_layers - 1)))
		self.embed_item_MLP = nn.Embedding(
				item_num, factor_num * (2 ** (num_layers - 1)))

		MLP_modules = []
		for i in range(num_layers):
			input_size = factor_num * (2 ** (num_layers - i))
			MLP_modules.append(nn.Dropout(p=self.dropout))
			MLP_modules.append(nn.Linear(input_size, input_size//2))
			MLP_modules.append(nn.ReLU())
		self.MLP_layers = nn.Sequential(*MLP_modules)

		if self.model in ['MLP', 'GMF']:
			predict_size = factor_num 
		else:
			predict_size = factor_num * 2
		self.predict_layer = nn.Linear(predict_size, 1)

		self._init_weight_()

	def _init_weight_(self):
		""" We leave the weights initialization here. """
		if not self.model == 'NeuMF-pre':
			nn.init.normal_(self.embed_user_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_user_MLP.weight, std=0.01)
			nn.init.normal_(self.embed_item_GMF.weight, std=0.01)
			nn.init.normal_(self.embed_item_MLP.weight, std=0.01)

			for m in self.MLP_layers:
				if isinstance(m, nn.Linear):
					nn.init.xavier_uniform_(m.weight)
			nn.init.kaiming_uniform_(self.predict_layer.weight, 
									a=1, nonlinearity='sigmoid')

			for m in self.modules():
				if isinstance(m, nn.Linear) and m.bias is not None:
					m.bias.data.zero_()
		else:
			# embedding layers
			self.embed_user_GMF.weight.data.copy_(
							self.GMF_model.embed_user_GMF.weight)
			self.embed_item_GMF.weight.data.copy_(
							self.GMF_model.embed_item_GMF.weight)
			self.embed_user_MLP.weight.data.copy_(
							self.MLP_model.embed_user_MLP.weight)
			self.embed_item_MLP.weight.data.copy_(
							self.MLP_model.embed_item_MLP.weight)

			# mlp layers
			for (m1, m2) in zip(
				self.MLP_layers, self.MLP_model.MLP_layers):
				if isinstance(m1, nn.Linear) and isinstance(m2, nn.Linear):
					m1.weight.data.copy_(m2.weight)
					m1.bias.data.copy_(m2.bias)

			# predict layers
			predict_weight = torch.cat([
				self.GMF_model.predict_layer.weight, 
				self.MLP_model.predict_layer.weight], dim=1)
			precit_bias = self.GMF_model.predict_layer.bias + \
						self.MLP_model.predict_layer.bias

			self.predict_layer.weight.data.copy_(0.5 * predict_weight)
			self.predict_layer.bias.data.copy_(0.5 * precit_bias)

	def forward(self, user, item):
		if not self.model == 'MLP':
			embed_user_GMF = self.embed_user_GMF(user)
			embed_item_GMF = self.embed_item_GMF(item)
			output_GMF = embed_user_GMF * embed_item_GMF
		if not self.model == 'GMF':
			embed_user_MLP = self.embed_user_MLP(user)
			embed_item_MLP = self.embed_item_MLP(item)
			interaction = torch.cat((embed_user_MLP, embed_item_MLP), -1)
			output_MLP = self.MLP_layers(interaction)

		if self.model == 'GMF':
			concat = output_GMF
		elif self.model == 'MLP':
			concat = output_MLP
		else:
			concat = torch.cat((output_GMF, output_MLP), -1)

		prediction = self.predict_layer(concat)
		return prediction.view(-1)


# main:

In [17]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 5.2 MB/s 


In [18]:
import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter

'''
import model
import config
import evaluate
import data_utils
'''

'\nimport model\nimport config\nimport evaluate\nimport data_utils\n'

In [19]:
parser={}
parser["lr"] = 0.001          #help="learning rate"
parser["dropout"] = 0.0       #help="dropout rate"
parser["batch_size"] = 256    #help="batch size for training"
parser["epochs"] = 20         #help="training epoches"
parser["top_k"] = 10          #help="compute metrics@top_k"
parser["factor_num"] = 32     #help="predictive factors numbers in the model"
parser["num_layers"] = 3      #help="number of layers in MLP model"
parser["num_ng"] = 4          #help="sample negative items for training"
parser["test_num_ng"] = 99    #help="sample part of negative items for testing"
parser["out"] = True          #help="save model or   not"
parser["gpu"] = "0"           #help="gpu card ID"

In [20]:
os.environ["CUDA_VISIBLE_DEVICES"] = parser["gpu"]
cudnn.benchmark = True

In [21]:
%cd ..
%cd ..
!pwd

/content/drive/MyDrive/MSc/codes/recommendation
/content/drive/MyDrive/MSc/codes
/content/drive/MyDrive/MSc/codes


In [22]:
############################## PREPARE DATASET ##########################
train_data, test_data, user_num ,item_num, train_mat = load_all()

# construct the train and test datasets
train_dataset = NCFData(
		train_data, item_num, train_mat, parser["num_ng"], True)
test_dataset = NCFData(
		test_data, item_num, train_mat, 0, False)
train_loader = data.DataLoader(train_dataset,
		batch_size=parser["batch_size"], shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,
		batch_size=parser["test_num_ng"]+1, shuffle=False, num_workers=0)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [23]:
########################### CREATE MODEL #################################
if config.model == 'NeuMF-pre':
	assert os.path.exists(config.GMF_model_path), 'lack of GMF model'
	assert os.path.exists(config.MLP_model_path), 'lack of MLP model'
	GMF_model = torch.load(config.GMF_model_path)
	MLP_model = torch.load(config.MLP_model_path)
else:
	GMF_model = None
	MLP_model = None

model = NCF(user_num, item_num, parser["factor_num"], parser["num_layers"], 
						parser["dropout"], config.model, GMF_model, MLP_model)
model.cuda()
loss_function = nn.BCEWithLogitsLoss()

if config.model == 'NeuMF-pre':
	optimizer = optim.SGD(model.parameters(), lr=lr)
else:
	optimizer = optim.Adam(model.parameters(), lr=lr)

In [24]:
writer = SummaryWriter() # for visualization

In [28]:
from tqdm import tqdm

In [ ]:
########################### TRAINING #####################################
count, best_hr = 0, 0
for epoch in range(parser["epochs"]):
	model.train() # Enable dropout (if have).
	start_time = time.time()
	train_loader.dataset.ng_sample()

	for user, item, label in tqdm(train_loader):
		try:
			user = user.cuda()
			item = item.cuda()
			label = label.float().cuda()

			model.zero_grad()
			prediction = model(user, item)
			loss = loss_function(prediction, label)
			loss.backward()
			optimizer.step()
			# writer.add_scalar('data/loss', loss.item(), count)
			count += 1
		except:
			break


	model.eval()
	HR, NDCG = metrics(model, test_loader, parser["top_k"])

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if parser["out"]:
			if not os.path.exists(config.model_path):
				os.mkdir(config.model_path)
			torch.save(model, 
				'{}{}.pth'.format(config.model_path, config.model))

print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))


  0%|          | 0/19418 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19418/19418 [02:45<00:00, 117.42it/s]


The time elapse of epoch 000 is: 00: 03: 18
HR: 0.637	NDCG: 0.368


100%|██████████| 19418/19418 [02:42<00:00, 119.21it/s]


The time elapse of epoch 001 is: 00: 03: 12
HR: 0.662	NDCG: 0.392


100%|██████████| 19418/19418 [02:43<00:00, 118.61it/s]


...

100%|██████████| 19418/19418 [02:41<00:00, 120.14it/s]


The time elapse of epoch 008 is: 00: 03: 08
HR: 0.693	NDCG: 0.423


100%|██████████| 19418/19418 [02:41<00:00, 119.98it/s]


The time elapse of epoch 009 is: 00: 03: 10
HR: 0.689	NDCG: 0.419


 94%|█████████▍| 18210/19418 [02:31<00:09, 123.40it/s]